In [ ]:
#음성코드 - 확인 필요
'''
def main():
    result = []
    menu_list = ['아메리카노', '카페라떼', '카라멜마끼야또', '밀크폼', '카푸치노', '카페모카', '바닐라라떼', '초코밀크', '카라멜밀크', '레몬에이드', '청포도에이드', '자몽에이드']
    ice_list = ["아이스", "차가운", "시원한"]
    hot_list = ["핫", "뜨거운"]

    okt = Okt()
    ade_list = ["에이드", "청포도에이드", "자몽에이드", "레몬에이드", "허니자몽블랙티"]
    add_custom_dictionary(okt, ade_list)

    stopwords = ['어', '그럼', '잔', '개', '이랑', '드릴게요', '주세요', '아', '줘', '음', '오', '그러면', '앗', '엇', '아하', '오호', '어머']

    # Use Google TTS to get a sentence
    tts_sentence = "바닐라라떼 하나 아이스로 주세요"
    tts = gTTS(tts_sentence, lang="ko")
    tts.save("tts_output.mp3")

    # Use Google STT to get the sentence from audio
    sentence = get_sentence_from_speech()
    while not sentence:
        sentence = get_sentence_from_speech()

    # Process the obtained sentence
    menu_info_list = get_menu_info_from_user(sentence, menu_list, okt, stopwords, ice_list, hot_list, ade_list)
    result.append({"원문": sentence, "주문": menu_info_list})

    # Print the collected results
    for order_info in result:
        print(order_info)

    # Create a DataFrame from the collected results
    df = create_dataframe(result)
    print(df)
'''

'\ndef main():\n    result = []\n    menu_list = [\'아메리카노\', \'카페라떼\', \'카라멜마끼야또\', \'밀크폼\', \'카푸치노\', \'카페모카\', \'바닐라라떼\', \'초코밀크\', \'카라멜밀크\', \'레몬에이드\', \'청포도에이드\', \'자몽에이드\']\n    ice_list = ["아이스", "차가운", "시원한"]\n    hot_list = ["핫", "뜨거운"]\n\n    okt = Okt()\n    ade_list = ["에이드", "청포도에이드", "자몽에이드", "레몬에이드", "허니자몽블랙티"]\n    add_custom_dictionary(okt, ade_list)\n\n    stopwords = [\'어\', \'그럼\', \'잔\', \'개\', \'이랑\', \'드릴게요\', \'주세요\', \'아\', \'줘\', \'음\', \'오\', \'그러면\', \'앗\', \'엇\', \'아하\', \'오호\', \'어머\']\n\n    # Use Google TTS to get a sentence\n    tts_sentence = "바닐라라떼 하나 아이스로 주세요"\n    tts = gTTS(tts_sentence, lang="ko")\n    tts.save("tts_output.mp3")\n\n    # Use Google STT to get the sentence from audio\n    sentence = get_sentence_from_speech()\n    while not sentence:\n        sentence = get_sentence_from_speech()\n\n    # Process the obtained sentence\n    menu_info_list = get_menu_info_from_user(sentence, menu_list, okt, stopwords, ice_list, hot_list, ade_list)\n 

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

Mounted at /content/drive


In [ ]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import time

In [ ]:
from gtts import gTTS
import speech_recognition as sr

def get_sentence_from_speech():
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Say something:")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        sentence = recognizer.recognize_google(audio, language="ko-KR")
        print("You said:", sentence)
        return sentence
    except sr.UnknownValueError:
        print("Sorry, could not understand audio.")
        return None

In [ ]:
import pandas as pd

def create_dataframe(result):
    # Initialize an empty list to store data
    data = []

    # Extract data from the 'result' list
    for order_info in result:
        original_sentence = order_info['원문']
        for order in order_info['주문']:
            menu = order['메뉴']
            quantity = order['개수']
            temperature = order['온도']
            data.append([original_sentence, menu, quantity, temperature])

    # Create a DataFrame from the collected data
    df = pd.DataFrame(data, columns=['원문', '메뉴', '개수', '온도'])

    return df

In [ ]:
import re
from konlpy.tag import Okt

def add_custom_dictionary(okt, word_list):
    for word in word_list:
        okt.pos(word, stem=True, norm=True)

def extract_menu_info(sentence, menu_list, okt, ice_list, hot_list, ade_list):
    menu_info_list = []

    for menu in menu_list:
        if menu in sentence:
            # 메뉴가 문장에 포함되어 있다면 해당 메뉴에 대한 정보 추출
            quantity = 1
            temperature = ""

            # 개수 찾기
            index = sentence.find(menu)
            quantity_match = re.search(r'\b\d+\b', sentence[:index])
            if quantity_match:
                quantity = int(quantity_match.group())

            # 아이스 여부 확인
            if any(ice_word in sentence for ice_word in ice_list):
                temperature = "아이스"

            # 핫 여부 확인
            if any(hot_word in sentence for hot_word in hot_list):
                temperature = "핫"

            if menu in ade_list:
                temperature = "아이스"

            menu_info_list.append({"메뉴": menu, "개수": quantity, "온도": temperature})

    return menu_info_list

In [ ]:
def find_ice_hot(sentence):

    ice_keywords = ["아이스", "차가운", "시원한"]
    hot_keywords = ["핫", "뜨거운", "따뜻한"]

    ice_pattern = re.compile('|'.join(ice_keywords))
    hot_pattern = re.compile('|'.join(hot_keywords))

    ice_match = ice_pattern.search(sentence)
    hot_match = hot_pattern.search(sentence)

    is_ice = bool(ice_match)
    is_hot = bool(hot_match)

    return is_ice, is_hot

In [ ]:
def get_menu_info_from_user(sentence, menu_list, okt, ice_list, hot_list, ade_list):
    menu_info_list = extract_menu_info(sentence, menu_list, okt, ice_list, hot_list, ade_list)

    while not menu_info_list:
        print(f"\n'{sentence}'는 잘못된 주문입니다.")
        print("메뉴가 없습니다. 다시 입력해주세요.")
        sentence = input("입력: ")
        menu_info_list = extract_menu_info(sentence, menu_list, okt, ice_list, hot_list, ade_list)

    for menu_info in menu_info_list:
        while not menu_info.get("온도"):
            print(f"\n'{sentence}'는 잘못된 주문입니다.")
            print(f"{menu_info['메뉴']}의 온도 정보가 없습니다. 핫과 아이스 중 어떤 걸로 드릴까요?")
            sentence_temperature = input("입력: ")

            is_ice, is_hot = find_ice_hot(sentence_temperature)

            if is_ice == True and is_hot == False:
              menu_info["온도"] = "아이스"
              break
            elif is_ice == False and is_hot == True:
              menu_info["온도"] = "핫"
              break
            else:
              print(f"{sentence_temperature} 에서 온도 정보를 찾지 못했습니다.")
              continue

    return menu_info_list

In [ ]:
def check_order_confirmation(order_info):
    # Print the order information
    print("\n<주문 내역>")
    print(f"원문: {order_info['원문']}")
    for order in order_info['주문']:
        print(f"메뉴: {order['메뉴']}, 개수: {order['개수']}, 온도: {order['온도']}")

    # Get user confirmation
    while True:
        user_response = input("\n주문이 맞습니까?(네/아니오): ").strip().lower()
        if user_response in ['네', '맞아', '응', '어', '맞아요', '네 맞아요', '맞습니다']:
            return True
        elif user_response in ['아니오', '아니야', '아니에요', '다시할래', '처음으로', '아닌데', '처음부터 다시', '처음부터 다시할래', '처음부터 다시 할래', '틀렸어', '틀렸는데', '틀렸습니다']:
            return False
        else:
            print("올바른 응답이 아닙니다. 다시 입력해주세요.")

In [ ]:
#미리 정의된 주문형 main 함수
def main_process_multiple_orders():
    result = []
    menu_list = ['아메리카노', '카페라떼', '카라멜마끼야또', '밀크폼', '카푸치노', '카페모카', '바닐라라떼', '초코밀크', '카라멜밀크', '레몬에이드', '청포도에이드', '자몽에이드', '자몽허니블랙티']
    ice_list = ["아이스", "차가운", "시원한"]
    hot_list = ["핫", "뜨거운", "따뜻한"]

    okt = Okt()
    ade_list = ["에이드", "청포도에이드", "자몽에이드", "레몬에이드", "허니자몽블랙티"]
    add_custom_dictionary(okt, ade_list)

    sentences = ["자몽에이드 한 잔 줘","바닐라라떼 하나 아이스로 줘", "카페라떼 핫으로 주세요", "아이스 아메리카노 하나 주세요", "청포도에이드 줘", "따뜻한 카라멜마끼야또 줘", "시원한 카페라떼 하나", "카페모카 따뜻한 거 하나 주세요", "핫 아메리카노 주세요", "아이스 카푸치노 하나 부탁합니다"]
    #sentences = ["아이스 아메리카노 하나 주세요"]    #"아메리카노 아이스랑 카페라떼, 레몬에이드 하나 주세요", "아메리카노 아이스랑 카페라떼, 레몬에이드 하나씩 주세요", "카페라떼랑 아메리카노 하나씩 주세요", "아메리카노랑 카페라떼 아이스로 하나씩 주세요", "핫 카페라떼 하나랑 아메리카노 하나 주세요", "아이스 아메리카노, 핫 카페라떼 하나씩 주세요", "아메리카노, 카페라떼 다 아이스로 주세요"]

    for sentence in sentences:
        menu_info_list = get_menu_info_from_user(sentence, menu_list, okt, ice_list, hot_list, ade_list)
        result.append({"원문": sentence, "주문": menu_info_list})

    # Print the collected results
    '''
    for order_info in result:
        print(order_info)

    # check order confirmation -> 그러나 현재 메인함수에서는 저장형 주문으로 진행하기 때문에 주문이 맞는지 확인하는 것을 제외함.
    for order_info in result:
        while True:
            # 주문 확인
            if check_order_confirmation(order_info):
                print("주문이 확인되었습니다.")
                break
            else:
                print("다시 주문해주세요.")
    '''
    for order_info in result:
        print(order_info)

    #df = create_dataframe(result)
    #print(df)

In [ ]:
#입력형(메뉴 하나 주문) main함수
def main_process_input_orders():
    result = []
    menu_list = ['아메리카노', '카페라떼', '카라멜마끼야또', '밀크폼', '카푸치노', '카페모카', '바닐라라떼', '초코밀크', '카라멜밀크', '레몬에이드', '청포도에이드', '자몽에이드', '자몽허니블랙티']
    ice_list = ["아이스", "차가운", "시원한"]
    hot_list = ["핫", "뜨거운", "따뜻한"]

    okt = Okt()
    ade_list = ["에이드", "청포도에이드", "자몽에이드", "레몬에이드", "허니자몽블랙티"]
    add_custom_dictionary(okt, ade_list)

    user_input = None
    while True:
        print("안녕하세요 비니입니다.\n메뉴 하나씩 주문 가능합니다. 주문하시겠습니까?(종료는 0 을 입력): ")
        user_input = input()

        if user_input == '0':
            break

        menu_info_list = get_menu_info_from_user(user_input, menu_list, okt, ice_list, hot_list, ade_list)
        result = [{"원문": user_input, "주문": menu_info_list}]

        for order_info in result:
              # 주문 확인
              if check_order_confirmation(order_info):
                  print("주문이 확인되었습니다.")
              else:
                  print("주문을 누락시키겠습니다. 다시 주문해주세요. 다음 주문으로 넘어가겠습니다.")
                  result.remove(order_info)

        if len(result) == 0:
            print("Result None")
        else:
            print(result)

        print("\n")
        #df = create_dataframe(result)
        #print(df)

In [ ]:
import time

if __name__ == "__main__":
    start_time = time.time()  # Record the start time
    main_process_multiple_orders()
    #main_process_input_orders()
    end_time = time.time()  # Record the end time

    print(f"Execute Time: {end_time-start_time}")

{'원문': '자몽에이드 한 잔 줘', '주문': [{'메뉴': '자몽에이드', '개수': 1, '온도': '아이스'}]}
{'원문': '바닐라라떼 하나 아이스로 줘', '주문': [{'메뉴': '바닐라라떼', '개수': 1, '온도': '아이스'}]}
{'원문': '카페라떼 핫으로 주세요', '주문': [{'메뉴': '카페라떼', '개수': 1, '온도': '핫'}]}
{'원문': '아이스 아메리카노 하나 주세요', '주문': [{'메뉴': '아메리카노', '개수': 1, '온도': '아이스'}]}
{'원문': '청포도에이드 줘', '주문': [{'메뉴': '청포도에이드', '개수': 1, '온도': '아이스'}]}
{'원문': '따뜻한 카라멜마끼야또 줘', '주문': [{'메뉴': '카라멜마끼야또', '개수': 1, '온도': '핫'}]}
{'원문': '시원한 카페라떼 하나', '주문': [{'메뉴': '카페라떼', '개수': 1, '온도': '아이스'}]}
{'원문': '카페모카 따뜻한 거 하나 주세요', '주문': [{'메뉴': '카페모카', '개수': 1, '온도': '핫'}]}
{'원문': '핫 아메리카노 주세요', '주문': [{'메뉴': '아메리카노', '개수': 1, '온도': '핫'}]}
{'원문': '아이스 카푸치노 하나 부탁합니다', '주문': [{'메뉴': '카푸치노', '개수': 1, '온도': '아이스'}]}
Execute Time: 0.02346038818359375
